## 29-31 July 2019 Melt Event at Summit Station, Greenland

In [1]:
def readNOAAsfcmet(year,month):
    from   datetime import datetime
    import pandas   as     pd
    
    # Read a surface meteorology file from NOAA's site at Summit Station, Greenland.
    df   = pd.read_csv('/Users/vonw/data/summit/sfcmet/noaamet/met_sum_insitu_1_obop_minute_' + str(year) + '_' + str(month).zfill(2) + '.txt', delim_whitespace=True, parse_dates=True, names=['site','year','month','day','hour','minute','wind direction','wind speed','wind steadiness factor','barometric pressure','temperature at 2 meters','temperature at 10 meters','temperature at tower top','relative humidity','precipitation intensity'])
    
    #  Convert string data to numeric values
    df['year'].apply(int);
    df['month'].apply(int);
    df['day'].apply(int);
    df['hour'].apply(int);
    df['minute'].apply(int);
    df['barometric pressure'].apply(float);
    df['wind direction'].apply(float);
    df['wind speed'].apply(float);
    df['wind steadiness factor'].apply(float);
    df['temperature at 2 meters'].apply(float);
    df['temperature at 10 meters'].apply(float);
    df['temperature at tower top'].apply(float);
    df['relative humidity'].apply(float);
    df['precipitation intensity'].apply(float);
    
    # Create a time index.
    time = [datetime(t[0],t[1],t[2],t[3],t[4],0) for t in zip(df.year,df.month,df.day,df.hour,df.minute)] 
    df.index = time
    
    return df

In [2]:
import pandas as pd
import numpy  as np
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.palettes import Viridis3
from bokeh.plotting import figure
from rad2bt import rad2bt

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
from readDMV import readDMV
d = '/Users/vonw/data/paeri/raw/AE'
c1_30 = readDMV(d + '190730' + '/' + '190730' + 'C1.RNC')
c1_31 = readDMV(d + '190731' + '/' + '190731' + 'C1.RNC')

In [5]:
noaa = readNOAAsfcmet(2019,7)

In [6]:
p1=figure(plot_width=900,
       plot_height=200,
       x_axis_type='datetime',
       x_axis_label='Time (UTC)',
       x_range=[pd.datetime(2019,7,30,0,0,0), pd.datetime(2019,8,1,0,0,0)],
       y_axis_label='Radiance [mW m-2 sr-1 (cm-1)-1]',
       title='962-cm-1 microwindow radiance, 30-31 July 2019')
p1.line(c1_30.time.values, c1_30.mean_rad[:,957].values, color=Viridis3[0])
p1.line(c1_31.time.values, c1_31.mean_rad[:,957].values, color=Viridis3[1])

p2=figure(plot_width=900,
       plot_height=200,
       x_range=p1.x_range,
       x_axis_type='datetime',
       x_axis_label='Time (UTC)',
       y_axis_label='Radiance [mW m-2 sr-1 (cm-1)-1]',
       title='Outside Temperatures, 30-31 July 2019')
p2.line(c1_30.time.values,c1_30.airNearBBsTemp.values-273.15, color='tomato', legend='airNearBBsTemp')
p2.line(c1_31.time.values,c1_31.airNearBBsTemp.values-273.15, color='tomato')
p2.line(c1_30.time.values,c1_30.outsideAirTemp.values-273.15, color=Viridis3[2], legend='outsideAirTemp')
p2.line(c1_31.time.values,c1_31.outsideAirTemp.values-273.15, color=Viridis3[2])
p2.line(noaa['2019-07-30'].index, noaa['2019-07-30']['temperature at 2 meters'].values, color=Viridis3[0], legend='NOAA 2-m Temp')
p2.line(noaa['2019-07-31'].index, noaa['2019-07-31']['temperature at 2 meters'].values, color=Viridis3[1])
p2.line([c1_30.time[0].values, c1_31.time[-1].values], [0., 0.], color='blue', line_dash='dashed')

p3=figure(plot_width=900,
       plot_height=200,
       x_range=p1.x_range,
       x_axis_type='datetime',
       x_axis_label='Time (UTC)',
       y_axis_label='Radiance [mW m-2 sr-1 (cm-1)-1]',
       title='Brightness Temperatures, 30-31 July 2019')
p3.line(c1_30.time.values, rad2bt(c1_30.wnum1[374:395].mean(), c1_30.mean_rad[:,374:395].mean(axis=1)).values-273.15, color=Viridis3[0], legend='670-680 cm-1')
p3.line(c1_31.time.values, rad2bt(c1_31.wnum1[374:395].mean(), c1_31.mean_rad[:,374:395].mean(axis=1)).values-273.15, color=Viridis3[1])
p3.line(c1_30.time.values, rad2bt(c1_30.wnum1[957], c1_30.mean_rad[:,957]).values-273.15, color=Viridis3[0], legend='962 cm-1')
p3.line(c1_31.time.values, rad2bt(c1_31.wnum1[957], c1_31.mean_rad[:,957]).values-273.15, color=Viridis3[1])
p3.line([c1_30.time[0].values, c1_31.time[-1].values], [0., 0.], color='blue', line_dash='dashed')

layout = gridplot([[p1], [p2], [p3]])
show(layout)

/Users/vonw/opt/anaconda3/envs/work/lib/python3.7/site-packages/xarray/core/computation.py:603: RuntimeWarning: invalid value encountered in log
  result_data = func(*input_data)


In [7]:
c1_30.wnum1[957]

<xarray.DataArray 'wnum1' ()>
array(961.40155029)
Coordinates:
    wnum1    float64 961.4

In [8]:
p4=figure(plot_width=900,
       plot_height=600,
       x_axis_label='Window Radiance [mW m-2 sr-1 (cm-1)-1]',
       y_axis_label='CO2 Brightness Temperature (K)',
       title='Window radiance vs. Outside Air Temperature, 30-31 July 2019')
p4.scatter(c1_30.mean_rad[:,957].values, c1_30.outsideAirTemp.values-273.15, color=Viridis3[0])
p4.scatter(c1_31.mean_rad[:,957].values, c1_31.outsideAirTemp.values-273.15, color=Viridis3[1])

show(p4)

In [66]:
p4=figure(plot_width=900,
       plot_height=600,
       x_axis_label='Window Radiance [mW m-2 sr-1 (cm-1)-1]',
       y_axis_label='CO2 Brightness Temperature (K)',
       title='Window radiance vs. CO2 Brightness Temperature, 30-31 July 2019')
p4.scatter(c1_30.mean_rad[:,957].values, rad2bt(c1_30.wnum1[374:395].mean(), c1_30.mean_rad[:,374:395].mean(axis=1)).values-273.15, color=Viridis3[0])
p4.scatter(c1_31.mean_rad[:,957].values, rad2bt(c1_31.wnum1[374:395].mean(), c1_31.mean_rad[:,374:395].mean(axis=1)).values-273.15, color=Viridis3[1])

show(p4)

## Turbulent Fluxes from the ICECAPS-ACE tower at TAWO

In [9]:
tf = pd.read_csv('/Users/vonw/work/projects/icecaps/analysis/Summit-Melt-Events/Turbulent Fluxes/All_flux_2019-05-30_to_2019-09-01', index_col='Dates', parse_dates=True)

In [10]:
windowRadiance = pd.DataFrame({'time': np.concatenate((c1_30.time.values, c1_31.time.values)), 'radiance': np.concatenate((c1_30.mean_rad[:,957].values, c1_31.mean_rad[:,957].values))})
windowRadiance.index = windowRadiance.time
windowRadiance = windowRadiance.drop(columns='time')
windowRadianceAve = windowRadiance.resample('15T').mean()

In [11]:
windowRadianceAve

,radiance
time,
2019-07-30 00:00:00,59.530643
2019-07-30 00:15:00,59.368721
2019-07-30 00:30:00,60.213573
2019-07-30 00:45:00,63.449619
2019-07-30 01:00:00,61.942673
...,...
2019-07-31 22:45:00,41.379578
2019-07-31 23:00:00,31.314003
2019-07-31 23:15:00,26.559822


In [12]:
# Sensible Heat
p1 = figure(plot_width=900,
            plot_height=200,
            x_axis_type='datetime',
            x_range=[pd.datetime(2019,7,30), pd.datetime(2019,8,1)],
            x_axis_label='Time (UTC)',
            y_axis_label='Flux (W m-2)',
            y_range=[-20, 20],
            title='Sensible Heat Fluxes from ICECAPS-ACE Tower, 2019 Melt Event')
p1.scatter(tf['2019-07-30 00':'2019-07-31 00'].index.values, tf['2019-07-30 00':'2019-07-31 00'].SHF.values, color=Viridis3[0])
p1.scatter(tf['2019-07-31 00':'2019-08-01 00'].index.values, tf['2019-07-31 00':'2019-08-01 00'].SHF.values, color=Viridis3[1])

# Latent Heat
p2 = figure(plot_width=900,
            plot_height=200,
            x_axis_type='datetime',
            x_range=p1.x_range,
            x_axis_label='Time (UTC)',
            y_axis_label='Flux (W m-2)',
            y_range=p1.y_range,
            title='Latent Heat Fluxes')
p2.scatter(tf['2019-07-30 00:00':'2019-07-31 00:00'].index.values, tf['2019-07-30 00:00':'2019-07-31 00:00'].LHF.values, color=Viridis3[0])
p2.scatter(tf['2019-07-31 00:00':'2019-08-01 00:00'].index.values, tf['2019-07-31 00:00':'2019-08-01 00:00'].LHF.values, color=Viridis3[1])

layout = gridplot([[p1], [p2]])
show(layout)


In [102]:
# Scatter plot of PAERI window radiance vs sensible heat flux
p1 = figure(plot_width=500,
            plot_height=500,
            x_axis_label='Time (UTC)',
            y_axis_label='Flux (W m-2)',
            y_range=[-20, 20],
            title='Dependence of Sensible Heat on Downwelling Radiance, July 2019 Melt Event')
p1.scatter(windowRadianceAve['2019-07-30 00':'2019-08-01 00'].values.reshape((192,)), tf['2019-07-30 00:00':'2019-07-31 23:45'].SHF.values, color=Viridis3[0])

show(p1)

## Radiosondes from Summit Station

In [13]:
from glob import glob

from bokeh.palettes import Category20_11
from bokeh.layouts import row
from bokeh.plotting import output_file

fns = glob('/Users/vonw/data/summit/radiosonde/raw/SMT_*_edt.txt')
fns.sort()

p1 = figure(plot_width=400, plot_height=600, y_range=[3, 13], title='Summit Radiosonde Profiles - July 2019 Melt Event')
for fn, color in zip(fns, Category20_11):
    snd = pd.read_table(fn, skiprows=8)
    name = fn[-23:-8]
    p1.line(snd['    Temp'], snd['  Height']/1000, line_width=2, color=color, legend=name)
p1.legend.location = "top_right"
p1.legend.click_policy="hide"

p2 = figure(plot_width=400, plot_height=600, y_range=p1.y_range)
for fn, color in zip(fns, Category20_11):
    snd = pd.read_table(fn, skiprows=8)
    name = fn[-23:-8]
    p2.line(snd['Humidity'], snd['  Height']/1000, line_width=2, color=color, legend=name)
p2.legend.location = "top_right"
p2.legend.click_policy="hide"

#output_file('/Users/vonw/work/projects/icecaps/analysis/Summit-Melt-Events/190729_190731/SummitSondes_20190729-20190802.html')
show(row(p1, p2))